<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#tf.nn.dynamic_rnn" data-toc-modified-id="tf.nn.dynamic_rnn-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>tf.nn.dynamic_rnn</a></span></li><li><span><a href="#参考资料" data-toc-modified-id="参考资料-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>参考资料</a></span></li></ul></div>

# tf.nn.dynamic_rnn

用一个小例子来说明 `tf.nn.dynamic_rnn` 的用法:

假设你的输入是 $inputs=[2, 20, 18]$,  其中 $2$ 为 batch_size, 20为截断最大长度, 128 为embedding_size. 可以看出只有两个样例. 假设第一个样例的截断长度为20, 第二个样子的截断长度为13, 剩下的用 `0-padding` 填充. 例如: 在NLP中不同句子的长度, 有的长, 有的短. 

dynamic_rnn 返回两个参数outputs, last_states, 其中, outputs是[2, 20, 128], 也就是每一个迭代隐状态的输出, last_states是由(c, h)组成的tuple, 均为[batch, 128]。

dynamic_rnn有一个参数:sequence_length, 这个参数来指定每个example的长度, 比如上面例子中sequence_langth:[20, 13], 表示第一个examle的长度为20, 第二个example的长度为13, 第二个参数的其余部分用`0-padding`, 对于第二个参数, 超过13以后padding就不计算了,其last_states将复制13步的last_state直到第20步, 而outputs中超过13步的结果将会被置零.

In [12]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

In [13]:
# 创建输入数据
# X Shape: [2, 10, 1]
X = np.random.randn(2, 10, 1)

# 第二个的截断长度只为6
X[1, 6:] = 0

# 指定截断长度
X_length = [10, 6]

# 建立基础LSTM
cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=3)

# 扩展RNN网络
outputs, last_states = tf.nn.dynamic_rnn(cell=cell, 
                                         inputs=X, 
                                         dtype=tf.float64, 
                                         sequence_length=X_length)

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    # 计算结果
    result_outputs, result_last_states = sess.run([outputs, last_states])
    
    # 打印运算结果
    print("result_outputs shape: ", np.array(result_outputs).shape)
    print("result_outputs: \n", result_outputs)
    print("\nresult_last_state shape: ", np.array(result_last_states).shape)
    print("result_last_states: \n", result_last_states)

result_outputs shape:  (2, 10, 3)
result_outputs: 
 [[[-0.01063851  0.01748171 -0.08379733]
  [-0.00340277 -0.00960592 -0.03479423]
  [-0.0253704   0.01941522 -0.18626434]
  [-0.02717365 -0.00720103 -0.18333199]
  [-0.02976138 -0.02103011 -0.15031428]
  [-0.04253727 -0.00632649 -0.20103233]
  [-0.04947873 -0.01396211 -0.21166135]
  [-0.05547664 -0.01783599 -0.21543861]
  [-0.03994731 -0.07020354 -0.13266371]
  [-0.04618341 -0.050701   -0.13769976]]

 [[ 0.02079382 -0.04478659  0.09032885]
  [ 0.03235999 -0.06676555  0.15557945]
  [ 0.01545323 -0.02326013  0.06964482]
  [ 0.0006022   0.00756315 -0.03478671]
  [-0.00551326  0.01206128 -0.0675198 ]
  [-0.00126666 -0.0071909  -0.02702726]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]]

result_last_state shape:  (2, 2, 3)
result_last_states: 
 LSTMStateTuple(c=array([[-0.09700437, -0.10569769, -0.29588657],
       [-0.00249645,

第一个样例输出了10个数据, 每个数据的的维度为[3]; 第二个样例, 也输出了10个数据, 不过后四个全部为0;

状态值的shape为[2, 2, 3]: 第一个2表示样例的个数, 第二个2表示(h, c), 第三个3表示(h, c)的维度为[3]

# 参考资料

1. 网络